## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Assiniboia,CA,2021-05-31 20:54:34,49.6334,-105.9845,78.71,14,100,14.72,overcast clouds
1,1,Vila Velha,BR,2021-05-31 20:54:13,-20.3297,-40.2925,78.75,85,0,5.01,clear sky
2,2,Port Blair,IN,2021-05-31 20:55:18,11.6667,92.7500,82.00,81,8,4.81,clear sky
3,3,Upernavik,GL,2021-05-31 20:47:47,72.7868,-56.1549,31.30,90,100,3.04,overcast clouds
4,4,Manakara,MG,2021-05-31 20:55:19,-22.1333,48.0167,69.66,93,34,6.80,light rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("Enter minimum temperature."))
max_temp = float(input("Enter maximum temperature."))

Enter minimum temperature.75
Enter maximum temperature.90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
sub_city_data_df = city_data_df.loc[
                    (city_data_df["Max Temp"] <= max_temp) & 
                    (city_data_df["Max Temp"] >= min_temp)]


In [5]:
# 4a. Determine if there are any empty rows.
sub_city_data_df.isnull().sum()

City_ID                0
City                   0
Country                0
Date                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
sub_city_data_df = sub_city_data_df.dropna()
sub_city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Assiniboia,CA,2021-05-31 20:54:34,49.6334,-105.9845,78.71,14,100,14.72,overcast clouds
1,1,Vila Velha,BR,2021-05-31 20:54:13,-20.3297,-40.2925,78.75,85,0,5.01,clear sky
2,2,Port Blair,IN,2021-05-31 20:55:18,11.6667,92.7500,82.00,81,8,4.81,clear sky
7,7,Bara,NG,2021-05-31 20:51:48,10.3744,10.7288,80.78,55,94,12.77,overcast clouds
14,14,Kapaa,US,2021-05-31 20:55:22,22.0752,-159.3190,82.38,70,34,3.00,scattered clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = sub_city_data_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Assiniboia,CA,78.71,overcast clouds,49.6334,-105.9845,
1,Vila Velha,BR,78.75,clear sky,-20.3297,-40.2925,
2,Port Blair,IN,82.00,clear sky,11.6667,92.7500,
7,Bara,NG,80.78,overcast clouds,10.3744,10.7288,
14,Kapaa,US,82.38,scattered clouds,22.0752,-159.3190,
18,Annau,TM,85.50,scattered clouds,37.8876,58.5160,
20,Road Town,VG,84.24,scattered clouds,18.4167,-64.6167,
23,Gaoual,GN,82.24,overcast clouds,11.7500,-13.2000,
30,Devils Lake,US,75.31,clear sky,48.1128,-98.8651,
33,Faanui,PF,78.48,scattered clouds,-16.4833,-151.7500,


In [10]:
#Set const. params for hotel search
params = {"radius": 5000, "type": "lodging", "key": g_key}

#Get hotel coord.
for index, row in hotel_df.iterrows():
        lat = row["Lat"]
        lng = row["Lng"]
        
        #Add to params
        params["location"] = f"{lat},{lng}"
        
        #Send gmaps request
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        hotels = requests.get(base_url, params = params).json()
        
        #Get first result if available
        try:
            hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
            print(str(index) + " " + hotels["results"][0]["name"])
        except (IndexError): 
            print("Hotel not found ... skipping.")

0 Canalta Hotel Assiniboia
1 Quality Suites Vila Velha
2 Welcomhotel By ITC Hotels, Bay Island, Port Blair
Hotel not found ... skipping.
14 Sheraton Kauai Coconut Beach Resort
18 Buzmein Sokak
20 ZINGARA 76ft Catamaran, full crew-all included
23 Hotel koliba
30 Holiday Inn Express Devils Lake, an IHG Hotel
33 Bora-Bora Pearl Beach Resort & Spa
36 مقبرة ال مجمل
37 Hilo Hawaiian Hotel
38 Hotel San Carlos
39 Hotel Mpampa
Hotel not found ... skipping.
47 Villa Enata
49 Bam Tourist Hotel
Hotel not found ... skipping.
62 صاحبة الفخامة
Hotel not found ... skipping.
80 Maunacaribe
88 Humacao Bed & Breakfast
89 Isles Sunset Lodge
95 Thunderbird Motor Inn
97 Nile Palace Hotel
Hotel not found ... skipping.
103 INN NEW HORIZON
Hotel not found ... skipping.
106 Cititel Penang
108 Railway motel
109 Hotel Sarah
111 JABIR HOTEL
116 La Villa Mahatsara
117 Grage Hotel Bengkulu
124 Mali arts
129 PT PPI Lamongan Tambak C
Hotel not found ... skipping.
135 Auberge Du Tékrour
141 Whispering Bamboo Cove Resor

In [16]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df.dropna()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Assiniboia,CA,78.71,overcast clouds,49.6334,-105.9845,Canalta Hotel Assiniboia
1,Vila Velha,BR,78.75,clear sky,-20.3297,-40.2925,Quality Suites Vila Velha
2,Port Blair,IN,82.00,clear sky,11.6667,92.7500,"Welcomhotel By ITC Hotels, Bay Island, Port Blair"
7,Bara,NG,80.78,overcast clouds,10.3744,10.7288,
14,Kapaa,US,82.38,scattered clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
...,...,...,...,...,...,...,...
697,Grenville,GD,80.82,overcast clouds,12.1167,-61.6167,Valley Breeze Guest House
702,Zhicheng,CN,75.76,light rain,30.2956,111.5047,Wangjiang Building
711,Cordoba,AR,77.85,clear sky,-31.4135,-64.1811,Hotel Sussex
712,Nalut,LY,86.95,few clouds,30.3333,10.8500,


In [17]:
# 8a. Create the output path
output_path = "Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the csv
hotel_df.to_csv(output_path, index_label="City_ID")

In [20]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}°F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [21]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center = (30, 31), zoom_level = 1.5)
heat_layer = gmaps.heatmap_layer(locations, weights = max_temp,
                                dissipating = False, 
                                max_intensity = 300,
                                point_radius = 4)
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))